In [1]:
from utils import SUPPORTED_MODELS
from transformers import AutoConfig
from optimum.intel.openvino import OVModelForCausalLM
from pathlib import Path
import openvino as ov
from transformers import AutoTokenizer

model_id = "red-pajama-3b-chat"
# models_dir = Path("c:/Users/nsavelye/workspace/projects/openvino_notebooks/notebooks/254-llm-chatbot")
models_dir = Path("./")
model_dir = models_dir / model_id / "FP16"
# model_dir = models_dir / model_id / "FP16_calibrated"
# model_dir = models_dir / model_id / "INT8_compressed_weights"
model_configuration = SUPPORTED_MODELS[model_id]

core = ov.Core()
# device = "CPU"
device = "GPU"

tok = AutoTokenizer.from_pretrained(models_dir / model_id / "RedPajama-INCITE-Chat-3B-v1", trust_remote_code=True)

ov_config = {'PERFORMANCE_HINT': 'LATENCY', 'NUM_STREAMS': '1', "CACHE_DIR": ""}
ov_model = OVModelForCausalLM.from_pretrained(model_dir, device=device, ov_config=ov_config,
                                              config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
                                              trust_remote_code=True)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


2023-11-20 23:48:44.611636: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 23:48:44.612732: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 23:48:44.633862: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 23:48:44.634317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 23:48:45.058125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
import importlib
import shutil
import partially_upcast_nodes_to_fp32
import model_upcast_utils
import main
importlib.reload(partially_upcast_nodes_to_fp32)
importlib.reload(main)


model = partially_upcast_nodes_to_fp32.partially_upcast_nodes_to_fp32(ov_model.model, main.get_inputs_for_calibration(
    ov_model, tok, model_configuration, "<human>: Which lakes are near Munich?\n<bot>:"), batch_size=50, verbose=True)
# model = model_upcast_utils.partially_upcast_nodes_to_fp32(ov_model.model, get_inputs_for_calibration("<human>: Which lakes are near Munich?\n<bot>:"))
calibrated_model_dir = Path(f"{model_dir}_calibrated")
ov.save_model(model, calibrated_model_dir / "openvino_model.xml")
shutil.copy(model_dir / "config.json", calibrated_model_dir / "config.json")
# ov_model = OVModelForCausalLM.from_pretrained(calibrated_model_dir, device=device, ov_config=ov_config,
#                                               config=AutoConfig.from_pretrained(calibrated_model_dir, trust_remote_code=True),
#                                               trust_remote_code=True)
ov_model.model = model
ov_model._original_model = model
ov_model.request = None
ov_model.compile()

 25%|██████████████████████████████▎                                                                                          | 1/4 [00:50<02:31, 50.58s/it]

Upcasted node __module.model.gpt_neox.layers.1.mlp.dense_4h_to_h/aten::linear/MatMul_702 with 0.10 rel2_diff_ratio 0.049910 and mean_rel_error 0.035067
Upcasted node __module.model.gpt_neox.layers.2.mlp.dense_4h_to_h/aten::linear/MatMul_937 with 0.10 rel2_diff_ratio 0.047837 and mean_rel_error 0.034047
Upcasted node __module.model.gpt_neox.layers.3.mlp.dense_4h_to_h/aten::linear/MatMul_1172 with 0.10 rel2_diff_ratio 0.048047 and mean_rel_error 0.034022
Upcasted node __module.model.gpt_neox.layers.4.mlp.dense_4h_to_h/aten::linear/MatMul_1407 with 0.10 rel2_diff_ratio 0.047386 and mean_rel_error 0.033766
Upcasted node __module.model.gpt_neox.layers.5.mlp.dense_4h_to_h/aten::linear/MatMul_1642 with 0.10 rel2_diff_ratio 0.048077 and mean_rel_error 0.033411
Upcasted node __module.model.gpt_neox.layers.6.mlp.dense_4h_to_h/aten::linear/MatMul_1877 with 0.10 rel2_diff_ratio 0.045523 and mean_rel_error 0.033319
Upcasted node __module.model.gpt_neox.layers.7.mlp.dense_4h_to_h/aten::linear/MatMul

 50%|████████████████████████████████████████████████████████████▌                                                            | 2/4 [01:40<01:40, 50.05s/it]

Upcasted node __module.model.gpt_neox.layers.8.mlp.dense_4h_to_h/aten::linear/MatMul_2347 with 0.10 rel2_diff_ratio 0.049399 and mean_rel_error 0.034660
Upcasted node __module.model.gpt_neox.layers.9.mlp.dense_4h_to_h/aten::linear/MatMul_2582 with 0.10 rel2_diff_ratio 0.048197 and mean_rel_error 0.033691
Upcasted node __module.model.gpt_neox.layers.10.mlp.dense_4h_to_h/aten::linear/MatMul_2817 with 0.10 rel2_diff_ratio 0.046725 and mean_rel_error 0.033831
Upcasted node __module.model.gpt_neox.layers.11.mlp.dense_4h_to_h/aten::linear/MatMul_3052 with 0.10 rel2_diff_ratio 0.044802 and mean_rel_error 0.033132
Upcasted node __module.model.gpt_neox.layers.12.mlp.dense_4h_to_h/aten::linear/MatMul_3287 with 0.10 rel2_diff_ratio 0.045763 and mean_rel_error 0.032555
Upcasted node __module.model.gpt_neox.layers.13.mlp.dense_4h_to_h/aten::linear/MatMul_3522 with 0.10 rel2_diff_ratio 0.049339 and mean_rel_error 0.035232
Upcasted node __module.model.gpt_neox.layers.14.mlp.dense_4h_to_h/aten::linear

 75%|██████████████████████████████████████████████████████████████████████████████████████████▊                              | 3/4 [02:30<00:50, 50.17s/it]

Upcasted node __module.model.gpt_neox.layers.16.mlp.dense_4h_to_h/aten::linear/MatMul_4227 with 0.10 rel2_diff_ratio 0.048858 and mean_rel_error 0.034349
Upcasted node __module.model.gpt_neox.layers.17.mlp.dense_4h_to_h/aten::linear/MatMul_4462 with 0.10 rel2_diff_ratio 0.049639 and mean_rel_error 0.034044
Upcasted node __module.model.gpt_neox.layers.18.mlp.dense_4h_to_h/aten::linear/MatMul_4697 with 0.10 rel2_diff_ratio 0.048738 and mean_rel_error 0.035155
Upcasted node __module.model.gpt_neox.layers.19.mlp.dense_4h_to_h/aten::linear/MatMul_4932 with 0.10 rel2_diff_ratio 0.050240 and mean_rel_error 0.035955
Upcasted node __module.model.gpt_neox.layers.20.mlp.dense_4h_to_h/aten::linear/MatMul_5167 with 0.10 rel2_diff_ratio 0.047837 and mean_rel_error 0.035032
Upcasted node __module.model.gpt_neox.layers.21.mlp.dense_4h_to_h/aten::linear/MatMul_5402 with 0.10 rel2_diff_ratio 0.048678 and mean_rel_error 0.034784
Upcasted node __module.model.gpt_neox.layers.22.mlp.dense_4h_to_h/aten::line

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:14<00:00, 48.61s/it]

Upcasted node __module.model.gpt_neox.layers.25.mlp.dense_4h_to_h/aten::linear/MatMul_6342 with 0.10 rel2_diff_ratio 0.050962 and mean_rel_error 0.035390
Upcasted node __module.model.gpt_neox.layers.26.mlp.dense_4h_to_h/aten::linear/MatMul_6577 with 0.10 rel2_diff_ratio 0.055138 and mean_rel_error 0.037367
Upcasted node __module.model.gpt_neox.layers.27.mlp.dense_4h_to_h/aten::linear/MatMul_6812 with 0.10 rel2_diff_ratio 0.050962 and mean_rel_error 0.036008
Upcasted node __module.model.gpt_neox.layers.28.mlp.dense_4h_to_h/aten::linear/MatMul_7047 with 0.10 rel2_diff_ratio 0.049219 and mean_rel_error 0.035956
Upcasted node __module.model.gpt_neox.layers.29.mlp.dense_4h_to_h/aten::linear/MatMul_7282 with 0.10 rel2_diff_ratio 0.040986 and mean_rel_error 0.030744
Upcasted node __module.model.gpt_neox.layers.30.mlp.dense_4h_to_h/aten::linear/MatMul_7517 with 0.10 rel2_diff_ratio 0.041647 and mean_rel_error 0.031767



Compiling the model to GPU ...


In [3]:
import importlib
import main
importlib.reload(main)

prompt = "Which lakes are near Munich?"

generation_kwargs = dict(
        max_new_tokens=100,
        temperature=0.1,
        do_sample=0.1 > 0.0,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.2
)


# print(run_generate(ov_model, prompt, model_configuration, **generation_kwargs))
for text in main.run_generate(ov_model, tok, prompt, model_configuration, **generation_kwargs):
    print(text, end="")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/home/guest/nsavelye/venvs/fp16_calibration/lib/python3.8/site-packages/optimum/intel/openvino/modeling_decoder.py:388: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)


Which lakes are near Munich?
<bot>: There is a total of five large and beautiful alpine lakes in the Bavarian Alps.  The largest lake, Lake Starnberg (Starnberger See) has an area of about 100 km2 with over 200 islands that can be reached by boat or on foot from two small towns: Obertrum/Trumersee and Unterach am Attersee which have both been popular destinations for tourists since Roman times due to their warm water springs. The second-largest